In [1]:
# !wget https://huggingface.co/datasets/mesolitica/kesalahan-tatabahasa/resolve/main/train.jsonl -O tatabahasa1.jsonl

In [2]:
# !wget https://huggingface.co/datasets/mesolitica/kesalahan-tatabahasa/resolve/main/train2.jsonl -O tatabahasa2.jsonl

In [3]:
describe = [
    {'class': 0,
     'Description': 'PAD',
     'salah': '',
     'betul': ''},
    {'class': 1,
     'Description': 'kesambungan subwords',
     'salah': '',
     'betul': '',
     },
    {'class': 2,
     'Description': 'tiada kesalahan',
     'salah': '',
     'betul': '',
     },
    {'class': 3,
     'Description': 'kesalahan frasa nama, Perkara yang diterangkan mesti mendahului "penerang"',
     'salah': 'Cili sos',
     'betul': 'sos cili',
     },
    {'class': 4,
     'Description': 'kesalahan kata jamak',
     'salah': 'mereka-mereka',
     'betul': 'mereka',
     },
    {'class': 5,
     'Description': 'kesalahan kata penguat',
     'salah': 'sangat tinggi sekali',
     'betul': 'sangat tinggi',
     },
    {'class': 6,
     'Description': 'kata adjektif dan imbuhan "ter" tanpa penguat.',
     'salah': 'Sani mendapat markah yang tertinggi sekali.',
     'betul': 'Sani mendapat markah yang tertinggi.',
     },
    {'class': 7,
     'Description': 'kesalahan kata hubung',
     'salah': 'Sally sedang membaca bila saya tiba di rumahnya.',
     'betul': 'Sally sedang membaca apabila saya tiba di rumahnya.',
     },
    {'class': 8,
     'Description': 'kesalahan kata bilangan',
     'salah': 'Beribu peniaga tidak membayar cukai pendapatan.',
     'betul': 'Beribu-ribu peniaga tidak membayar cukai pendapatan',
     },
    {'class': 9,
     'Description': 'kesalahan kata sendi',
     'salah': 'Umar telah berpindah daripada sekolah ini bulan lalu.',
     'betul': 'Umar telah berpindah dari sekolah ini bulan lalu.',
     },
    {'class': 10,
     'Description': 'kesalahan penjodoh bilangan',
     'salah': 'Setiap orang pelajar',
     'betul': 'Setiap pelajar.',
     },
    {'class': 11,
     'Description': 'kesalahan kata ganti diri',
     'salah': 'Pencuri itu telah ditangkap. Beliau dibawa ke balai polis.',
     'betul': 'Pencuri itu telah ditangkap. Dia dibawa ke balai polis.',
     },
    {'class': 12,
     'Description': 'kesalahan ayat pasif',
     'salah': 'Cerpen itu telah dikarang oleh saya.',
     'betul': 'Cerpen itu telah saya karang.',
     },
    {'class': 13,
     'Description': 'kesalahan kata tanya',
     'salah': 'Kamu berasal dari manakah ?',
     'betul': 'Kamu berasal dari mana ?',
     },
    {'class': 14,
     'Description': 'kesalahan tanda baca',
     'salah': 'Kamu berasal dari manakah .',
     'betul': 'Kamu berasal dari mana ?',
     },
    {'class': 15,
     'Description': 'kesalahan kata kerja tak transitif',
     'salah': 'Dia kata kepada saya',
     'betul': 'Dia berkata kepada saya',
     },
    {'class': 16,
     'Description': 'kesalahan kata kerja transitif',
     'salah': 'Dia suka baca buku',
     'betul': 'Dia suka membaca buku',
     },
    {'class': 17,
     'Description': 'penggunaan kata yang tidak tepat',
     'salah': 'Tembuk Besar negeri Cina dibina oleh Shih Huang Ti.',
     'betul': 'Tembok Besar negeri Cina dibina oleh Shih Huang Ti',
     },
]

kesalahan = {}
for d in describe[2:]:
    kesalahan[d['class']] = d['Description']
    
len(kesalahan)

16

In [4]:
keys = set(kesalahan.keys())
choice = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}

In [6]:
import random

In [7]:
import json
from tqdm import tqdm

qa = []

for f in ['tatabahasa1.jsonl', 'tatabahasa2.jsonl']:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            
            if len(l[0]) < 5:
                continue
            if len(l[0]) != len(l[1]):
                continue
            for i in range(len(l[1])):
                if l[1][i][1] not in {3}:
                    t = l[1][i][0]
                    if l[1][i][1] == 2:
                        if t.istitle() or t.isupper():
                            continue
                        if len(t) < 3:
                            continue

                        if random.random() > 0.1:
                            continue

                    if random.random() > 0.3:
                        continue

                    k = [l[1][k][0] for k in range(i)] + [f'({l[1][i][0]})'] + [l[1][k][0] for k in range(i + 1, len(l[1]), 1)]
                    choices = random.sample(keys - {l[1][i][1]}, 3) + [l[1][i][1]]
                    random.shuffle(choices)
                    jawapan = [choice[no] for no, c in enumerate(choices) if c == l[1][i][1]]
                    choices = {choice[no]: kesalahan[c] for no, c in enumerate(choices)}
                    q = {
                        'soalan': f'Apakah kesalahan tatabahasa untuk ({l[1][i][0]})',
                        'konteks': ' '.join(k),
                        'choice': choices,
                        'jawapan': jawapan[0],
                        'pembetulan': l[0][i][0]
                    }
                    qa.append(q)
                
len(qa)

497050it [00:09, 54756.71it/s]
497079it [00:09, 52829.75it/s]


1462807

In [8]:
with open('kesalahan-tatabahasa-choice.jsonl', 'w') as fopen:
    for q in qa:
        fopen.write(f'{json.dumps(q)}\n')